__Вопросы для повторения__:
* Что такое шаблоны в С++ и зачем они нужны?
* Сколько здесь будет скомпилировано функций `print`? Какие?

```c++
template<typename T>
void print(const T& x)
{
    std::cout << x << std::endl;
}

int main()
{
    print(5);
    print(6);
    print(5.0);
    print(6.0f);
    print("Dobrynia");
    print("Nikitich");
    print("Alesha");
    print("Popovich");
    print(std::string("Ilya"));
    print(std::string("Muromec"));
}
```

* Сколько раз компилируется шаблон?
* Как быть с ODR?
* Плюсы и минусы шаблонов?
* Пример с лекции:

```c++
template<int N, typename T>
T add_value(T x)
{
    return x + N;
}

int a = add_value<5>(100);
```

* Что здесь происходит? Сколько будет скомпилировано структур F?

```c++
template<int N>
struct F
{
    static const int value = N * Factorial<N - 1>;
};

template<>
struct F<0>
{
    static const int value = 1;
};

int main()
{
    return F<5>::value;
}
```

* Что в этом примере?

```c++
template<typename T>
const T& min(const T a, const T b)
{
    if (a < b)
        return a;
    else
        return b;
}

int main()
{
    const std::string& s = min<std::string>("string 1", "string 2");
    std::cout << s << std::endl;
}
```

* Скомпилируется ли такой код?

```c++
template<typename T>
struct Vector3
{
    T x;
    T y;
    T z;
    
    Vector3<T> operator *(const float value)
    {
        return { x * value, y * value, z * value };
    }
};

std::string f()
{
    Vector3<std::string> v;
    return v.x
}
```

* А такой?

```c++
template<typename T>
struct Vector3
{
    T x;
    T y;
    T z;

    template<typename U>
    Vector3<T> operator *(const U& value)
    {
        return { x * value, y * value, z * value };
    }
};

std::string f()
{
    Vector3<std::string> v;
    return v.x
}
```

<br />

### Лекция 6. Контейнеры и итераторы

https://en.cppreference.com/w/cpp/container

https://apprize.info/c/optimized/10.html

Что такое контейнер и что такое итератор общими словами:
* контейнер - хранилище объектов одного типа
* итератор - "ключик" к конкретному объекту в контейнере, возможно, позволяющий "обходить" контейнер

```c++
// контейнер целых чисел
std::vector<int> v = {1, 2, 3, 4, 5};

// итератор, указывающий на нулевой элемент в контейнере:
std::vector<int>::iterator it = v.begin();
// auto it = v.begin();  // или так, компилятор сам выведет тип

++it;  // теперь it указывает на первый элемент в контейнере

*it = 42;

std::cout << v[1]; // # 42
v[1] = 55;
std::cout << *it; // # 55
```

__Вопросы__:
* с какими контейнерами мы уже имели дело в курсе?

<details>
<summary>Подсказка</summary>
<p>

`std::string`

</p>
</details>

* какие ещё контейнеры вы знаете?
* владеет ли контейнер объектами в нём? (про что этот вопрос?)

<br />

##### Стандартные контейнеры: последовательности

Памятка: обратить _особое_ внимание на внутреннюю организацию, `sizeof`, детали реализации, стоимость операций.

<br />

https://en.cppreference.com/w/cpp/container/array

Массив:
* compile-time размер
* объекты размещены непосредственно в `std::array`
* объекты размещены последовательно

```c++
// example
template<typename T, size_t N>
class array
{
private:
    T data_[N];

    ...;    
};
```

Важнейшее свойство `std::array`, которое выделяет его на фоне всех других контейнеров - отсутствие аллокаций внутри контейнера:

```c++
void func()
{
    // 5 целых чисел будут отмотаны НА СТЕКЕ, никаких вызовов malloc
    std::array<int, 5> arr;
    arr[2] = 0;
    arr[3] = 0;
    
    // добро пожаловать в динамическую память, malloc-free
    // и прочие радости медленного кода 
    std::vector<int> v = {1, 2, 3, 4, 5};
    
    // ну оооооооок, в динамической памяти
    auto* p = new std::array<int, 5>();
    ...
}
```

Стоимость:
* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

<br />

https://en.cppreference.com/w/cpp/numeric/valarray

* фиксированный размер в runtime (но можно менять через `resize`)
* выделение памяти на куче
* объекты размещены последовательно
* оптимизирован под мат.операции (в плане интерфейса и реализации)

```c++
float f(unsigned n)
{
    std::valarray<float> v1(1.f, n);
    std::valarray<float> v2(2.f, n);
    
    // рассмотрим два примера использования,
    // что в них происходит, какой из них быстрее?
    
    // ex.1
    auto v3 = v1 * 2.f + v2;
    
    // ex.2
    auto v4 = v1;
    v4 *= 2.f;
    v4 += v2;
    
    return v4.sum();
}
```

Как организован внутри?

Стоимость:
* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

<br />

https://en.cppreference.com/w/cpp/container/vector

http://www.cvl.isy.liu.se:82/education/graduate/opencv/pres.pdf

* меняющийся размер
* выделение памяти на куче
* объекты размещены последовательно

```c++
int f()
{
    std::vector<int> v1 = {1, 2, 3, 4, 5};
    v1.push_back(6);
    v1.push_back(7);
    v1.push_back(8);
    
    v1.clear();
    v1.push_back(9);
    v1.insert(v1.begin(), 1);
    
    return v1.back();
}
```

![](vector_internals.png)

Эта история будет вечной...

```c++
// ex 1
std::vector<int> v1;
for (int i = 0; i < N; ++i)
    v1.push_back(i);
// Вопрос: сложность алгоритма в кол-ве копирований
// Вопрос: сложность алгоритма в кол-ве аллокаций
    
// ex 2
std::vector<int> v2;
v2.reserve(N);
for (int i = 0; i < N; ++i)
    v1.push_back(i);
// Вопрос: сложность алгоритма в кол-ве копирований
// Вопрос: сложность алгоритма в кол-ве аллокаций
```

Стоимость:
* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка в начало - ???
* вставка в середину - ???
* вставка в конец - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

Специальное исключение:

```c++
std::vector<bool> bv;
```

<br />

https://en.cppreference.com/w/cpp/container/deque

* меняющийся размер
* выделение памяти на куче
* объекты размещены последовательными блоками, а блоки раскиданы по памяти

Нарисовать как хранится `deque`: блоки + переходы (bookkeeping)

![](deque_internals.png)

Размер блока зависит от реализации: 8 times the object size on 64-bit libstdc++; 16 times the object size or 4096 bytes, whichever is larger, on 64-bit libc++

```c++
std::deque<int> d = {1, 2, 3, 4 ,5};
        
d.push_back(6);
d.push_front(0);

d.pop_back();
d.pop_front();
```

Стоимость:
* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка в начало - ???
* вставка в середину - ???
* вставка в конец - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

<br />

https://en.cppreference.com/w/cpp/container/list

* меняющийся размер
* выделение памяти на куче отдельно под каждый элемент
* оптимизирован под вставку / удаление элементов в произвольном месте

![](list_internals.png)

```c++
std::list<int> l = { 7, 5, 16, 8 };
l.push_back(4);
l.push_front(3);

l.sort();
```

Стоимость:

* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка в начало - ???
* вставка в середину - ???
* вставка в конец - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

<br />

https://en.cppreference.com/w/cpp/container/forward_list

Вариант `std::list` с урезанным функционалом, но более дешёвый по памяти.

![](forwardlist_internals.jpg)

```c++
std::forward_list<int> f = {1, 2, 3, 4, 5};
f.push_front(0);
f.insert_after(f.begin(), 1);
```

Стоимость:

* расходы памяти на i-ый элемент - ???
* доступ до i-го элемента - ???
* поиск по значению - ???
* вставка в начало - ???
* вставка в середину - ???
* вставка в конец - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

<br />

##### Стандартные контейнеры: ассоциативные

https://en.cppreference.com/w/cpp/container/map

https://en.cppreference.com/w/cpp/container/set

https://en.cppreference.com/w/cpp/container/multimap

https://en.cppreference.com/w/cpp/container/multiset

Отображения / множества, основанные на порядке элементов. Обычно реализуются в виде красно-чёрных деревьев по ключам.

Главное требование к ключам - они должны иметь порядок с требованиями Compare (strict weak ordering + equivalence https://en.cppreference.com/w/cpp/named_req/Compare):
* либо определён `operator<` для ключей, удовлетворящий требованиям Compare
* либо специфицирована пользовательская функция сравнения, удовлетворящия требованиям Compare

Замечание: если сравнение не удовлетворяет требованиям Compare - UB

Важное свойство: порядок обхода объектов - отсортированы по ключу.

```c++
// создание отображения
std::map<int, std::string> id_to_name = {
    {73, "Balin"},
    {42, "Dwalin"},
    {55, "Gloin"}
};

// добавление элементов:
id_to_name[53] = "Gimli";

// изменение:
id_to_name[53] = "Torin";

// особенность map-подобных контейнеров: operator[] создаёт элемент, если его там нет
const std::string name = id_to_name[54]; // # name == "", но в id_to_name теперь есть такой элемент: {54, ""}
        
// перебор элементов (порядок!!)
{
    // так код не пишите, это до С++11
    for(std::map<int, std::string>::iterator it = id_and_name.begin(); it != id_and_name.end(); ++it)
        std::cout << it->first << " " << it->second << std::endl;

    // так код не пишите, только для иллюстрации
    for (const std::pair<const int, std::string>& id_and_name : id_to_name)
        std::cout << id_and_name.first << " " << id_and_name.second << std::endl;
    
    // если не разрешён С++17, то хотя бы так
    for (const auto& id_and_name : id_to_name)
        std::cout << id_and_name.first << " " << id_and_name.second << std::endl;
    
    // начиная с С++17:
    for (const auto& [id, name] : id_to_name)
        std::cout << id << " " << name << std::endl;
}

// поиск элемента в массиве
{
    // до С++11
    std::map<int, std::string>::iterator it = id_to_name.find(42);
    if (it != id_to_name.end())
        std::cout << it->second;
    
    // после C++11
    auto it = id_to_name.find(42);
    if (it != id_to_name.end())
        std::cout << it->second;
}
```

```c++
std::set<std::string> animals = {"cat", "dog", "dolphin"};
animals.insert("elephant");
animals.erase("dragon");

// !порядок
for (const auto& animal : animals)
    std::cout << animal << std::endl;
```

![](set_internals.jpg)

Задание оператора сравнения:

```c++
// пользовательский тип
struct Point
{
    float x, y;
};

// определяем понятие меньше через лексикографическое сравнение
bool operator < (const Point& l, const Point& r) noexcept
{
    return std::tie(l.x, l.y) < std::tie(r.x, r.y);
}

// теперь можно делать set/map:
std::set<Point> points;
```

```c++
// пользовательский тип
struct Person
{
    std::string name;
    int age;
};

// не хотим определять operator< или хотим отличную от него сортировку в set (почему-то)
struct LessPersionByAge
{
    bool operator()(const Person& l, const Person& r) const noexcept
    {
        return std::tie(l.age, l.name) < std::tie(r.age, r.name);
    }
};

// теперь можно:
std::set<Person, LessPersionByAge> people;    

// продвинутый вариант, почему он работает - рассмотрим, когда будем работать с лямбдами:
auto less_persion_by_age = [](const Person& l, const Person& r) {
    return std::tie(l.age, l.name) < std::tie(r.age, r.name);
};
set::set<Person, decltype(less_persion_by_age)> people;
```

Стоимость:

* расходы памяти на элемент - ???
* поиск по значению - ???
* вставка - ???
* удаление - ???
* сортировка - ???
* sizeof - ???

<br />

https://en.cppreference.com/w/cpp/container/unordered_map
    
https://en.cppreference.com/w/cpp/container/unordered_set
    
https://en.cppreference.com/w/cpp/container/unordered_multimap

https://en.cppreference.com/w/cpp/container/unordered_multiset

Отображения / множества, основанные на значении хеш-функции от элементов.

Обычно реализуются в виде массива списков (из-за специфики требований их сложно реализовать через открытую адресацию).

(Не забыть рассказать про load_factor и rehashing)

![](unordered_internals.jpg)

В качестве функции хеширования по умолчанию испольуется `std::hash`, но можно и задавать собственную функцию хеширования (аналогично собственной функции сравнения для `std::map`)

```c++
std::unordered_map<int, std::string> id_to_name = {
    {73, "Balin"},
    {42, "Dwalin"},
    {55, "Gloin"}
};
                
// порядок перечисления объектов не гарантирован!
for (const auto& [id, name] : id_to_name)
    std::cout << id;
    
std::unordered_set<std::string> animals = {"cat", "dog", "dolphin"};
                
// порядок перечисления объектов не гарантирован!
for (const auto& animal : animals)
    std::cout << animal;
```

Заполнение unordered-контейнера, обратить внимание на load_factor + rehash:

```c++
std::unordered_set<int> ids;
for (int i = 0; i < 1000; ++i)
    ids.insert(generate_id());
// Вопрос: сложность алгоритма в кол-ве копирований
// Вопрос: сложность алгоритма в кол-ве аллокаций

std::unordered_set<int> ids;
ids.reserve(1000);
for (int i = 0; i < 1000; ++i)
    ids.insert(generate_id());
// Вопрос: сложность алгоритма в кол-ве копирований
// Вопрос: сложность алгоритма в кол-ве аллокаций
```

__Замечание__: если слегка ослабить требования к `unordered_*` контейнерам, например, разрешить протухать итераторам при вставке новых элементов, то hash-based отображения можно значительно (в разы) ускорить, что и было сделано сторонними реализациями.

Например, реализация от google: https://www.youtube.com/watch?v=ncHmEUmJZf4

И море, море их. Толковый обзорный доклад с деревом решений "какой когда использовать":
https://www.youtube.com/watch?v=M2fKMP47slQ

Стоимость:

* расходы памяти на элемент - ???
* поиск по значению - ???
* вставка - ???
* удаление - ???
* сортировка - ???
* sizeof - ???